In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from scipy.spatial.distance import cosine
from tqdm import trange

# 설정
n_qubits = 3
shots = 1024
iterations = 10
noise_level = 0.3

sim = Aer.get_backend('qasm_simulator')

base_angles = [0.25 * np.pi, 0.5 * np.pi, 0.75 * np.pi]

def build_circuit(angles):
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i, theta in enumerate(angles):
        qc.ry(theta, i)
    qc.measure(range(n_qubits), range(n_qubits))
    return qc

def run_circuit(angles):
    qc = build_circuit(angles)
    result = sim.run(qc, shots=shots).result()
    return result.get_counts()

def counts_to_vector(counts):
    keys = sorted([f"{i:03b}" for i in range(8)])
    vec = np.array([counts.get(k, 0) for k in keys], dtype=float)
    return vec / vec.sum()

def similarity(v1, v2):
    return 1 - cosine(v1, v2)

results = []
for _ in trange(iterations):
    angles_a = base_angles
    counts_a = run_circuit(angles_a)
    vec_a = counts_to_vector(counts_a)

    angles_b = [theta + np.random.normal(0, noise_level) for theta in base_angles]
    counts_b = run_circuit(angles_b)
    vec_b = counts_to_vector(counts_b)

    angles_c = [np.random.uniform(0, np.pi) for _ in range(n_qubits)]
    counts_c = run_circuit(angles_c)
    vec_c = counts_to_vector(counts_c)

    sim_ab = similarity(vec_a, vec_b)
    sim_bc = similarity(vec_b, vec_c)
    g_ex = sim_ab / (sim_bc + 1e-6)

    results.append((sim_ab, sim_bc, g_ex))

print(f"{'A-B 유사도':>12} | {'B-C 유사도':>12} | {'G_ex':>6}")
print("-" * 36)
for ab, bc, gx in results:
    print(f"{ab:12.4f} | {bc:12.4f} | {gx:6.2f}")
